<a href="https://colab.research.google.com/github/Pravinoraon/capstone/blob/main/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-addons numpy matplotlib pillow scikit-learn
!pip install flask flask-cors requests numpy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, Model
from tensorflow.keras.applications import ResNet50
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)


2.18.0


In [ ]:
pip install --upgrade tensorflow


In [ ]:
dataset_dir = '/content/drive/MyDrive/Data Set'
img_size = (224, 224)
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
images, labels = [], []
for folder in ['Healthy', 'Diseased']:
    class_folder = os.path.join(dataset_dir, folder)
    class_index = 0 if folder == 'Healthy' else 1
    for img in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img)
        if img.endswith('.jpg') or img.endswith('.png'):  # Ensure correct file type
            images.append(img_path)
            labels.append(class_index)

images = np.array(images)
labels = np.array(labels)


In [ ]:
k = 2
kf = KFold(n_splits=k, shuffle=True, random_state=42)
fold_no = 1
accuracies = []


In [ ]:
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings

    def build(self, input_shape):
        self.kernel = self.add_weight(name='capsule_kernel',
                                      shape=(input_shape[-1], self.num_capsule * self.dim_capsule),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs):
        u_hat = tf.matmul(inputs, self.kernel)
        u_hat = tf.reshape(u_hat, (-1, self.num_capsule, self.dim_capsule))
        b = tf.zeros_like(u_hat[:, :, 0])

        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=1)
            s = tf.reduce_sum(tf.expand_dims(c, -1) * u_hat, axis=1)
            v = tf.nn.l2_normalize(s, axis=-1)
            b += tf.reduce_sum(u_hat * tf.expand_dims(v, 1), axis=-1)

        return v


In [ ]:
def create_hybrid_model():
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False

    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Reshape((-1, 1))(x)

    x = CapsuleLayer(num_capsule=10, dim_capsule=16, routings=3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
for train_idx, val_idx in kf.split(images, labels):
    print(f'\nFold {fold_no} of {k}')

    train_images, val_images = images[train_idx], images[val_idx]
    train_labels, val_labels = labels[train_idx], labels[val_idx]

    # Convert labels to strings
    train_df = pd.DataFrame({'filename': train_images, 'class': train_labels.astype(str)})
    val_df = pd.DataFrame({'filename': val_images, 'class': val_labels.astype(str)})

    # Check for class imbalance
    if len(train_df['class'].unique()) < 2 or len(val_df['class'].unique()) < 2:
        print(f"Warning: Fold {fold_no} has class imbalance. Skipping this fold.")
        fold_no += 1
        continue  # Skip to the next fold

    train_gen = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='filename',
        y_col='class',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )

    val_gen = datagen.flow_from_dataframe(
        dataframe=val_df,
        x_col='filename',
        y_col='class',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )

    model = create_hybrid_model()
    history = model.fit(train_gen, validation_data=val_gen, epochs=10)

    val_acc = history.history['val_accuracy'][-1]
    accuracies.append(val_acc)
    fold_no += 1


Fold 1 of 2

Fold 2 of 2


In [ ]:
print("\nCross-Validation Results:")
for i, acc in enumerate(accuracies):
    print(f"Fold {i+1}: Accuracy = {acc:.4f}")

print(f"\nAverage Accuracy: {np.mean(accuracies):.4f}")




Cross-Validation Results:

Average Accuracy: nan


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
